In [ ]:
#!jupyter nbconvert --to=python DSi_WOA_Comparison_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class DSicomp:
    '''
    class DSicomp(runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 WOAvar='i_an',mapproj='pc',savefig=False,layerwise=False)
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 WOAvar='i_an',
                 mapproj='pc',
                 savefig=False,
                 layerwise=False,depth_array=[]):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array

        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        #from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from load_interp_WOA_python3 import WOAdata
        from plot_Taylor_normalized import plt_Taylor_norm

        # load FESOM data -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = [self.fyear, self.lyear]

        # check variables
        NCfesom = self.resultpath + '/DSi.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM DSi {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'DSi [mmol Si m$^{-3}$]' 

        # load data -------------------------------------------------------------------------------------
        DSifesom = pf.get_data(resultpath, "DSi", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)


        # load WOA data  -------------------------------------------------------------------------------------
        dsi_int = WOAdata(self.runname,self.resultpath,self.meshpath,self.ncpath,self.WOAvar, get_overview=False).woa_int    
        #print(np.shape(din_int))
        
        labelwoa = 'World Ocean Atlas DSi'
        unitwoa = 'DSi [$\mu$mol Si m$^{-3}$]' #i_an:units = "micromoles_per_liter"
        
        print('***\nUnit WOA: '+unitwoa+'\nUnit FESOM: '+unitfesom+'\n***')

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        dsi_int_ma = np.copy(dsi_int)
        dsi_int_ma[DSifesom == 0] = 0

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(True):
                    print('\nInput depth = {0}, plotting at depth = {1} m\nFESOM min = {2}, max = {3}\nWOA min = {4}, max = {5}'.format(
                        d,plot_depth,
                        np.nanmin(DSifesom[:,i]),np.nanmax(DSifesom[:,i]),
                        np.nanmin(dsi_int_ma[:,i]),np.nanmax(dsi_int_ma[:,i])))

                pf.plot(mesh, [dsi_int_ma[:,i],DSifesom[:,i], dsi_int_ma[:,i]-DSifesom[:,i]], 
                        rowscol= (1,3),
                        levels = (-6,120,64),
                        units=unitwoa + '\n at depth = {0} m'.format(plot_depth), 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        titles=[labelwoa, labelfesom, 'WOA - FESOM'],figsize = (20,20)
                       )

                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig('{0}{1}_DSi_WOA_at{2}m_{3}to{4}.png'.format(
                        self.savepath, self.runname, plot_depth, str(years[0]), str(years[1])), 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False) 
            
            # statistics  -------------------------------------------------------------------------------------
            # preparation of datasets
            if np.isnan(np.min(dsi_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DSifesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)
                
                title = 'Taylor Diagram for DSi at {0} m'.format(plot_depth)
                plt_Taylor_norm(dsi_int[i,:],DSifesom[i,:],mask=True,title=title)


                if(self.savefig==True):
                    plt.savefig('{0}{1}_DSi_WOA_at{2}m_{3}to{4}.png'.format(
                        self.savepath, self.runname, plot_depth, str(years[0]), str(years[1])), 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False) 
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            
            DSifesom_mean = np.mean(DSifesom, axis = 1)
            dsi_int_ma_mean = np.mean(dsi_int_ma, axis = 1)

            print('\nPlotting DSi as mean over depth\nFESOM min = {0}, max = {1}\nWOA min = {2}, max = {3}'.format(
                    np.nanmin(DSifesom_mean),np.nanmax(DSifesom_mean),
                    np.nanmin(dsi_int_ma_mean),np.nanmax(dsi_int_ma_mean)))
    
            fig_data = pf.plot(mesh, [dsi_int_ma_mean,DSifesom_mean, dsi_int_ma_mean-DSifesom_mean], 
                    rowscol= (3,1),
                    levels = (-6,120,64),
                    units=unitwoa, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    titles=['World Ocean Atlas: Interpolated DSi for initialization', 
                            'FESOM Dissolved Silicate ({0}-{1})'.format(first_year,last_year),
                            'Initialization fields minus FESOM'],
                    figsize = (20,20)
                   )
        
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'DSi_WOA'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
                
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(dsi_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DSifesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

            title = 'Taylor Diagram for DSi \n(mean over depth)'
            plt_Taylor_norm(dsi_int_ma_mean,DSifesom_mean,mask=True,title=title)
            
            if False: #old
                # get statistics only from ocean gridpoints (same mask assumption as above)
                ind_stat = np.where(DSifesom_mean != 0)

                taylor_stats1 = sm.taylor_statistics(dsi_int_ma_mean[ind_stat],DSifesom_mean[ind_stat])
                sdev = np.array([taylor_stats1['sdev'][0], taylor_stats1['sdev'][1]])
                crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
                ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])

                label = ['Observation', 'Model']# at {0} m'.format(plot_depth)]

                fig_stat = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
                sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                                      titleOBS = 'observation', markerLabel = label,
                                      markerLabelColor = 'c',
                                      markerColor = 'c', markerLegend = 'on',
                                      #tickRMS = range(0,5,1), 
                                      tickRMSangle = 135.0,
                                      colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                                      titleRMS = 'off', #tickSTD = range(0,10,2 ),
                                      #axismax = 10.0, 
                                      colSTD = 'b', styleSTD = '-.',
                                      widthSTD = 1.0, titleSTD = 'on',
                                      colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                                      titleCOR = 'on')
                plt.title('Taylor Diagram for DSi \n(mean over depth)', loc='right')  
                
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'DSi_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
        


In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo5'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    htmlname     =  runid + '.html'
    htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    #modulepath = '/home/ollie/mozeisin/py_recom/GlobalAssessment'
    #modulepath   = '/home/ollie/jhauck/py_fesom/modules/'

    #obspath      = '/home/ollie/mseifert/py_fesom/CreateRunFiles/Initialization/Output/'

    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1948
    last_year  = 1950

    # WOA ------------------------------------------------------------------------------------    
    ncpath = '/work/ollie/projects/MarESys/evaluation/woa13_all_i00_01.nc'
    #!ncdump -h $ncpath

    # now test:
    test = DSicomp(runid,resultpath,savepath,meshpath,ncpath,first_year,last_year,layerwise=False,savefig=False)